In [56]:
import requests
import lxml
import os

In [84]:
def search(query,size,synonym='YES'):
  #change synonym to no otherwise yes is the default
  if size<=1000:
    headers = {'Content-type': 'application/x-www-form-urlencoded'}
    payload = {'query': query,'resultType':'core','pageSize':1000,'synonym':synonym}
    r = requests.post('https://www.ebi.ac.uk/europepmc/webservices/rest/searchPOST', data=payload,headers=headers)
    return r.content
  if size>1000:
    headers = {'Content-type': 'application/x-www-form-urlencoded'}
    payload = {'query': query,'format':format,'resultType':'core','pageSize':1000}
    r = requests.post('https://www.ebi.ac.uk/europepmc/webservices/rest/searchPOST', data=payload,headers=headers)
    xmlschema_doc = lxml.etree.XML(r.content)
    cursor=xmlschema_doc.xpath('//nextCursorMark')
    nextCursorMark=cursor[0].text
    headers = {'Content-type': 'application/x-www-form-urlencoded'}
    payload = {'query': query,'format':format,'resultType':'core','cursorMark':nextCursorMark,'pageSize':1000}
    r2 = requests.post('https://www.ebi.ac.uk/europepmc/webservices/rest/searchPOST', data=payload,headers=headers)
    return r.content+r2.content

def makefiles(searchvariable,size):
  xmlschema_doc = lxml.etree.XML(searchvariable)
  PMC=[]
  for a in xmlschema_doc.xpath('//fullTextId'):
    PMC.append(a.text)

  for i in PMC[0:size]:
    r = requests.get(f"https://www.ebi.ac.uk/europepmc/webservices/rest/{i}/fullTextXML")
    root = lxml.etree.fromstring(r.text)
    tree = lxml.etree.ElementTree(root)
    if not os.path.isdir(os.path.join(str(os.getcwd()),'papers')):
      os.makedirs(os.path.join(str(os.getcwd()),'papers'))
    tree.write(os.path.join(str(os.getcwd()),'papers',f"{i}.xml"))

In [80]:
#type search query in this form
a=search('covid',50)

#pass in this query to makefiles along with the number of files you want to save
makefiles(a,50)